In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder , StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error , mean_squared_error

dataset_path = r'C:\Users\Admin\Documents\AIO---262-Trinh-Xd\Module 3\Week 4\Housing.csv'
df = pd.read_csv(dataset_path)
print(df)

        price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0    13300000  7420         4          2        3      yes        no       no   
1    12250000  8960         4          4        4      yes        no       no   
2    12250000  9960         3          2        2      yes        no      yes   
3    12215000  7500         4          2        2      yes        no      yes   
4    11410000  7420         4          1        2      yes       yes      yes   
..        ...   ...       ...        ...      ...      ...       ...      ...   
540   1820000  3000         2          1        1      yes        no      yes   
541   1767150  2400         3          1        1       no        no       no   
542   1750000  3620         2          1        1      yes        no       no   
543   1750000  2910         3          1        1       no        no       no   
544   1750000  3850         3          1        2      yes        no       no   

    hotwaterheating aircond

# Xử lý dữ liệu categorical:

In [2]:
'''
Để có thể thực hiện bất kì tính toán nào, ta cần phải số hóa toàn
bộ giá trị trong bảng dữ liệu. Nhận thấy có một số cột trong bảng dữ liệu có dạng string, ta sẽ
phải đổi toàn bộ các giá trị này về dạng số. Đầu tiên, ta kiểm tra các cột có kiểu dữ liệu là Object
như sau:
'''
categorical_cols = df.select_dtypes(include=['object']).columns.to_list()
print(categorical_cols)

['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']


In [3]:
'''
Sau khi đã xác định được đối tượng cần xử lý, ta sẽ sử dụng OrdinalEncoder() để chuyển đổi
chúng thành dạng số như sau:
'''
ordinal_encoder = OrdinalEncoder()
encoded_catergorical_cols = ordinal_encoder.fit_transform(
    df[categorical_cols]
)

encoded_catergorical_cols_df = pd.DataFrame(
    encoded_catergorical_cols,
    columns=categorical_cols
)

numerical_df = df.drop(categorical_cols, axis = 1)

encoded_df = pd.concat(
    [numerical_df, encoded_catergorical_cols_df], axis = 1
)
print(encoded_df)

        price  area  bedrooms  bathrooms  stories  parking  mainroad  \
0    13300000  7420         4          2        3        2       1.0   
1    12250000  8960         4          4        4        3       1.0   
2    12250000  9960         3          2        2        2       1.0   
3    12215000  7500         4          2        2        3       1.0   
4    11410000  7420         4          1        2        2       1.0   
..        ...   ...       ...        ...      ...      ...       ...   
540   1820000  3000         2          1        1        2       1.0   
541   1767150  2400         3          1        1        0       0.0   
542   1750000  3620         2          1        1        0       1.0   
543   1750000  2910         3          1        1        0       0.0   
544   1750000  3850         3          1        2        0       1.0   

     guestroom  basement  hotwaterheating  airconditioning  prefarea  \
0          0.0       0.0              0.0              1.0     

# Chuẩn hoá bộ dữ liệu

In [4]:
'''
Để việc tính toán thuận lợi, ta tiến hành chuẩn hóa toàn bộ giá trị
trong bộ dữ liệu sử dụng StandardScaler() như sau:
'''

normalizer = StandardScaler()

dataset_arr = normalizer.fit_transform(encoded_df)
print(dataset_arr)

[[ 4.56636513  1.04672629  1.40341936 ...  1.4726183   1.80494113
  -1.40628573]
 [ 4.00448405  1.75700953  1.40341936 ...  1.4726183  -0.55403469
  -1.40628573]
 [ 4.00448405  2.21823241  0.04727831 ... -0.67906259  1.80494113
  -0.09166185]
 ...
 [-1.61432675 -0.70592066 -1.30886273 ... -0.67906259 -0.55403469
   1.22296203]
 [-1.61432675 -1.03338891  0.04727831 ... -0.67906259 -0.55403469
  -1.40628573]
 [-1.61432675 -0.5998394   0.04727831 ... -0.67906259 -0.55403469
   1.22296203]]


# Tách dữ liệu X, y:

In [5]:
'''
Khi đã hoàn tất các bước tiền xử lý, lúc này ta sẽ tách dữ liệu ban đầu thành
hai biến X, y đại diện cho các đặc trưng và nhãn. Nhận thấy cột đầu tiên, price, là nhãn của bộ
dữ liệu, ta sẽ tiến hành tách X, y ra như sau:
'''
X, y = dataset_arr [: , 1:] , dataset_arr [: , 0]

# Chia tập dữ liệu train, val:

In [6]:
'''
Dựa vào bộ dữ liệu gốc, ta cần chia thành hai tập dữ liệu con,
một dùng cho việc huấn luyện mô hình và một cho việc đánh giá mô hình. Ở đây, ta sẽ chia theo
tỷ lệ 7:3 và tham số ngẫu nhiên random_state = 1:'''
test_size = 0.3
random_state = 1
is_shuffle = True
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size= test_size,
    random_state= random_state,
    shuffle= is_shuffle
)

# Huấn luyện mô hình:

In [7]:
'''
Ta thực hiện huấn luyện mô hình với bộ dữ liệu train. Để huấn luyện
mô hình Random Forest, các bạn sẽ sử dụng RandomForestRegressor():'''
regressor = RandomForestRegressor(
    random_state = random_state
)
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

In [8]:
# Để huấn luyện mô hình AdaBoost, các bạn sẽ sử dụng AdaBoostRegressor():
regressor = AdaBoostRegressor(
    random_state = random_state
)
regressor.fit( X_train , y_train )

AdaBoostRegressor(random_state=1)

In [9]:
# Để huấn luyện mô hình Gradient Boosting, các bạn sẽ sử dụng GradientBoostingRegressor():
regressor = GradientBoostingRegressor (
    random_state = random_state
)
regressor.fit(X_train , y_train )

GradientBoostingRegressor(random_state=1)

# Đánh giá mô hình:

In [12]:
'''
Để biết được mô hình đã huấn luyện có hoạt động tốt trên các mẫu dữ liệu
mới hay không, ta sẽ đánh giá thông qua tập val. Đầu tiên, ta cho mô hình đã huấn luyện thực
hiện dự đoán trên toàn bộ tập val:
'''
y_pred = regressor.predict(X_val)
print(y_pred)

[ 0.95205743  0.08542726  1.32252954  0.58276196 -0.30234557 -0.77410498
 -0.61956404  1.48970226 -1.01361197 -0.27536753 -0.40053959 -0.68114807
 -0.1381536  -0.26108812  0.92270112  1.79460478 -0.1916804  -0.46613825
  0.96006387 -1.03606011 -0.21133677  0.37104205  0.30087142 -0.68569388
  0.03391174  1.5326558  -1.14116168  2.10137857  1.25153789 -0.58337395
  1.42310912  1.03833552 -0.32401392  0.05065641  0.54345203 -0.26631342
  0.61470176 -1.05907106 -0.00791383  0.81345691 -0.30924226 -0.85083452
 -0.66139733 -0.28012753  0.95477084 -0.55920368  0.18285793 -0.97848245
 -1.43370009 -0.91182063 -0.11578759 -0.29103835 -1.17974253 -0.56743554
  0.18844201 -0.17875393 -0.82594517 -0.13893977 -0.31519932 -0.18678381
  1.27691756  0.89259305 -1.05181401 -0.1823532   0.01169717 -1.28378089
  0.04289818 -0.89755458 -0.28188392 -0.21289128  0.62860378  0.19347619
  0.49869    -1.04905537  0.31232125 -0.82349888 -0.78878993  1.15507193
  0.68950176 -0.57346236  0.3802518  -0.29039134  1

In [13]:
mae = mean_absolute_error (y_val , y_pred )
mse = mean_squared_error (y_val , y_pred )

print ('Evaluation results on validation set :')
print (f'Mean Absolute Error: {mae}')
print (f'Mean Squared Error: {mse}')

Evaluation results on validation set :
Mean Absolute Error: 0.4516626127750995
Mean Squared Error: 0.39610445936979427
